In [1]:
def distv3d(halo, center):
    norm = np.sqrt(np.square(center['vx'] - halo['vx']) + 
                   np.square(center['vy'] - halo['vy']) + 
                   np.square(center['vz'] - halo['vz']))
    return norm

def distv(halo, center):
    try:
        vx, vy, vz = center['vx'], center['vy'], center['vz']
    except:
        vx, vy, vz = center['vxc'], center['vyc'], center['vzc']
        
    try:
        vhx, vhy, vhz = halo['vx'], halo['vy'], halo['vz']
    except:
        vhx, vhy, vhz = halo['vxc'], halo['vyc'], halo['vzc']
    
    norm = np.sqrt(np.square(vx - vhx) + 
                   np.square(vy - vhy) + 
                   np.square(vz - vhz))
    return norm


def dist(halo, center):
    try:
        cx, cy, cz = center['x'], center['y'], center['z']
    except:
        cx, cy, cz = center['xc'], center['yc'], center['zc']
        
    try:
        hx, hy, hz = halo['x'], halo['y'], halo['z']
    except:
        hx, hy, hz = halo['xc'], halo['yc'], halo['zc']
    norm = np.sqrt(np.square(cx - hx) + 
                   np.square(cy - hy) + 
                   np.square(cz - hz))
    return norm 

def match_gal_hal_tree(gt, ht):
    nout = 187
    dt = 3 # compare progenitor at dt ago.
    
    gal_now = gt[gt["nout"]==nout]
    hal_now = ht[ht["nout"]==nout]
    
    gal_before = gt[gt["nout"]==nout-dt]
    hal_before = ht[ht["nout"]==nout-dt]    
    
    dominant = 0.1 # matched one should have less error by this amount or smaller 
                    # compared to the second best matched one.
    
    abs_tol_pos = 5e-5 # Position absolute tolerance [in code unit?]
    abs_tol_vel = 10   # velocity absolute tolerance [in kms?]
    
    for gal in gal_now:
        dd = dist(hal_now, gal)
        vv = distv(hal_now, gal)
        d_sort = np.argsort(dd)
        v_sort = np.argsort(vv)
        if (dd[d_sort[0]] < dominant * dd[d_sort[1]]) and (dd[d_sort[0]] < abs_tol_pos) and \
        (vv[v_sort[0]] < dominant * vv[v_sort[1]]) and (vv[v_sort[0]] < abs_tol_vel):
            gal['hosthalo'] = allhal.data['id'][d_sort[0]]
            i0.append(i)
            newhals[i] = allhal.data[d_sort[0]]
        else:
            atree = tree.atree(gt)
            prg = atree[dt]
            for gal2 in gal_before:
                dd = dist(hal_now, gal2)
                vv = distv(hal_now, gal2)
                d_sort = np.argsort(dd)
                v_sort = np.argsort(vv)

def get_comp_dist(hal_now, gal, nreturn=5, fpos=1.0, fvel=1.0):
    """Measure 6D distance and return Nreturn closest entries"""
    dd = dist(hal_now, gal)
    vv = distv(hal_now, gal)
    dd_q1 = np.percentile(dd,10)
    vv_q1 = np.percentile(vv,10)
    comp_dist = np.sqrt(fpos * np.square(dd/dd_q1) + fvel * np.square(vv/vv_q1))
    ind_sort = np.argsort(comp_dist)
    return comp_dist[ind_sort[:nreturn]], hal_now[ind_sort[:nreturn]]

def before_to_now(htdata, hals, dt):
    out = []
    for hal in hals:
        atree_hal = ctu.extract_main_tree_full(htdata, idx=hal['id'])
        out.append(atree_hal[hal['nout'] + dt])
    return out

def now_to_before(htdata, hals, dt):
    """
    progenitor of current halos.
    If does not exist, give -1
    """
    out =[]
    for hal in hals:
        idx = hal['id']
        try:
            atree_hal = ctu.extract_main_tree(htdata, idx=idx)
            out.append(atree_hal[dt])
        except:
            print("broken tree")
            out.append(-1)        
            
    return np.array(out)

def gal_hal_pair(cluster, gt, ht, info,
                    dominant = 0.4,
                    abs_tol_pos = 1e-2,
                    abs_tol_vel = 100,
                    nout_fi = 187,
                    dts = [3,5],
                    ids=None,
                    filter_dup=False,
                    verbose=False,
                    skip_tree=False,
                    fpos=1.0, fvel=1.0):
    """
        fpos, fvel:
            weight of position and velocity in comp_dist.
    """

    gal_now = gt.data[gt.data["nout"]==nout_fi]
    if ids is not None:
        from utils.match import match_list_ind
        i_good_gal = mtc.match_list_ind(gal_now["Orig_halo_id"], ids)
        gal_now = gal_now[i_good_gal]
    hal_now = ht.data[ht.data["nout"]==nout_fi]

    # exclude galaxies with too short tree.
    gal_ok = ctu.check_tree_complete(gt.data, nout_fi - max(dts), nout_fi, gal_now["id"], idx=True)

    hal_3 = ht.data[ht.data["nout"]==nout_fi - 3]
    hal_5 = ht.data[ht.data["nout"]==nout_fi - 5]
    hal_this_list = [hal_3, hal_5]
    
    comp_dists=[]
    #comp_dists.append(comp_dist)
    result = []
    mhal_result = []
    dist_error = []

    i_gal_ok = []
    ok_gals = []
    for igal, gal in enumerate(gal_now):
        if gal['id'] not in gal_ok:
            print("ID: {},  Too short tree".format(gal["id"]))
            continue
        else:
            i_gal_ok.append(igal)
            if verbose: print("Galaxy {} has a fine tree. Good to go.".format(gal["id"]))
            # Halo descendant - galaxy descendant
            comp_dist, good_hals_now = get_comp_dist(hal_now, gal, nreturn=5, fpos=fpos, fvel=fvel)
            # halo must be more massive than the galaxy
            matches=[]
            good_hals_now = good_hals_now[good_hals_now["m"] > gal["m"]]
            # Sometimes no halo larger than the galaxy is found in the vicinity.
            # but... why??
            try:
                if len(good_hals_now) == 0:
                    continue
            except:
                print(igal, comp_dist, good_hals_now)
            
            matches.append(good_hals_now["Orig_halo_id"])
            
            if skip_tree:
                ok_gals.append(gal)
                result.append(good_hals_now[0])
                continue
                
            # Halo progenitor - galaxy progenitor
            atree = ctu.extract_main_tree(gt.data, idx=gal['id'])            
            for idt, dt in enumerate([3,5]):
                hal_this = hal_this_list[idt]
                gal_this = atree[dt]
                
                # 10 closest galaxies.
                comp_dist_this, good_hals_this = get_comp_dist(hal_this, gal_this, nreturn=10)
                # halo must be more massive than the galaxy.
                good_hals_this = good_hals_this[good_hals_this["m"] > gal_this["m"]]
                
                # progenitors of the 'good' halo candidates
                good_hals_prgsthis = now_to_before(ht.data, good_hals_now, dt)
                
                i_good = []
                i_good_prg=[]
                try:
                    for i, ghthis in enumerate(good_hals_this['Orig_halo_id']):
                        # Why halo id at now and before are compared?? 
                        if ghthis in good_hals_prgsthis["Orig_halo_id"]:
                            #i_good_prg.append(i)
                            i_good.append(np.where(good_hals_prgsthis["Orig_halo_id"] == ghthis)[0][0])
                except:
                    #print(idt, dt, good_hals_this, good_hals_this.dtype, good_hals_this["Orig_halo_id"])
                    print(idt, dt, good_hals_prgsthis.dtype)
                    print(good_hals_prgsthis)
                    print(good_hals_now)
                    print(good_hals_prgsthis["Orig_halo_id"])
                if verbose: print(i_good)
                matches.append(good_hals_now["Orig_halo_id"][i_good])
            try:
                if matches[0][0] == matches[1][0] == matches[2][0]:
                    matched = matches[0][0]
                    result.append(good_hals_now[0])
                    ok_gals.append(gal)
            except:
                if verbose:
                    print(matches)
                pass


    result = np.array(result)
    ok_gals = np.array(ok_gals)

    print( "Out of {} galaxies, matched {} galaxies.".format(len(gal_now), len(result)))
    
    if ~filter_dup:
        return result, ok_gals
    
    # filter duplicates
    unq, unq_idx, unq_cnt = np.unique(result["Orig_halo_id"], return_inverse=True, return_counts=True)
    cnt_mask = unq_cnt > 1
    cnt_idx, = np.nonzero(cnt_mask)
    idx_mask = np.in1d(unq_idx, cnt_idx)
    idx_idx, = np.nonzero(idx_mask)
    srt_idx = np.argsort(unq_idx[idx_mask])
    dup_idx = np.split(idx_idx[srt_idx], np.cumsum(unq_cnt[cnt_mask])[:-1])

    # Remove smaller duplicates and leave the largest galaxy.
    remove_inds=[]
    for dup in dup_idx:
        ind_all = np.full(len(dup), True, dtype=bool)
        #print(ind_all)
        imax = np.argmax(ok_gals["m"][dup])
        ind_all[imax] = False
        remove_inds.extend(dup[ind_all])

    remove_inds = np.array(remove_inds)
    inds_ok = np.full(len(result), True, dtype=bool)
    inds_ok[remove_inds] = False


    # load each galaxy and measure stellar mass
    return result[inds_ok], ok_gals[inds_ok]


In [2]:
def hal_gal_by_gatalog(hcat, gcat, ids=None, fpos=1.0, fvel=0.0):
    hd = hcat.data
    gd = gcat.data
    if ids is not None:
        from utils.match import match_list_ind
        i_good_gal = mtc.match_list_ind(gd["id"], ids)
        gd = gd[i_good_gal]

    gd = gd[np.argsort(gd["m"])][::-1] # Sort in decending stellar mass order.

    i_gal_ok=[]
    matched_g=[]
    matched_h=[]

    for igal, gal in enumerate(gd):
        i_gal_ok.append(igal)

        comp_dist, good_hals = get_comp_dist(hd, gal, nreturn=3, fpos=fpos, fvel=fvel)
        #good_hals_now = good_hals_now[good_hals_now["m"] > gal["m"]]

        try:
            if len(good_hals) == 0:
                print(comp_dist)
                continue
        except:
            print(igal, comp_dist, good_hals)

        matched_h.append(good_hals[0])
        matched_g.append(gal)

    return np.hstack(matched_h), np.hstack(matched_g)

good_hals, good_gals = gal_hal_pair(cluster, gt, ht, info,
                                     ids = missing,
                                     dominant = 0.4,
                                     abs_tol_pos = 1e-2,
                                     abs_tol_vel = 100,
                                     nout_fi = 187,
                                     dts = dts,
                                     verbose=True,
                                     skip_tree=True,
                                     fpos=1.0, fvel=1)

ax = draw.pp.pp_halo(good_hals, 400, edgecolor='blue', rscale=1e-3)
#ax.pp_hal_meta.region["zr"] = (0, 1e9)
#ax.pp_hal_meta.region["ranges"][2] = (0, 1e9)
print("_______________________")
draw.pp.pp_halo(good_gals, 400, edgecolor='green', rscale=1e-3, ax=ax)
plt.show()

거리만으로 했을 때 어느정도 회복 가능. 그러나 아직도 안 되는 것들 있음. 
헤일로가 없는 것인가? 아니면 있는데 못 찾는 것인가? 

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import tree.halomodule as hmo
import pickle
from analysis.MajorMinorAccretion_module import *
import draw
import load
import utils.match as mtc
wdir = '/home/hoseung/Work/data/'

In [4]:
cd '/home/hoseung/Work/data/'

/home/hoseung/Work/data


In [5]:
# '_' at the tail...?
prg_final = pickle.load(open(wdir + "main_prgs_final_augmented_5_10_0.5_0.5_0.5_37_0.01_filtered.pickle", "rb"))

How do I define centrals and satellites? 

In [6]:
clusters = np.unique([gal.cluster for gal in prg_final])

In [ ]:
for cluster in clusters[6:]:
    sclu = str(cluster).zfill(5)
    wdir_clu = wdir + sclu +'/'
    hcat = hmo.Halo(base=wdir_clu, nout=187)
    gcat = hmo.Halo(base=wdir_clu, nout=187, is_gal=True)
    
    gal_this_cluster = [gal.data[0] for gal in prg_final if gal.cluster == cluster]
    gal_this_cluster = np.hstack(gal_this_cluster)
    mh, mg = hal_gal_by_gatalog(hcat, gcat, ids=gal_this_cluster["id"])

    #print(len(gal_this_cluster), len(np.unique(mh)))

    ax = draw.pp.pp_halo(mh, 400, color_field="level")
    #print("_______________________")
    draw.pp.pp_halo(mg, 400, edgecolor='green', ax=ax)
    ax.set_aspect('equal', 'datalim')
    ax.set_title("{}  out of {} matched".format(len(np.unique(mh)), len(gal_this_cluster)))
    plt.savefig(wdir + 'figs/hal_gal_match/' + sclu + "_hal_gal_match.png")
    plt.close()
    

In [49]:
mh["level"]

array([1, 1, 2, 1, 3, 2, 2, 2, 2, 2, 1, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 1, 2, 2, 2, 3, 2, 3, 1, 4, 2, 1, 2, 2, 2, 2, 2, 1, 2,
       2, 2, 2, 3, 2, 2, 3, 3, 2, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2,
       2, 1, 2, 2, 2, 2, 2, 1, 2, 2, 2, 1, 1, 2, 2, 2, 1, 2, 1, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 3, 2, 3, 2, 2, 2, 2, 2, 3, 4, 2, 3, 2, 2, 2, 2, 2, 2, 3, 3,
       2, 2, 2, 4, 2, 3, 2, 2, 2, 2, 2, 2, 4, 2, 2, 3, 2, 1, 4, 2, 2, 3, 3,
       3, 2, 4, 2, 2, 2, 2, 2, 2, 2, 3, 3, 2, 2, 2, 3, 1, 3, 2, 2, 1, 3, 1,
       2], dtype=int32)

8 galaxies with the same host halo. 
DM halos totally destructed? 

In [43]:
ax = draw.pp.pp_halo(hcat.data, 400, color_field="level")
print("_______________________")
draw.pp.pp_halo(mg, 400, edgecolor='green', ax=ax)
plt.show()

MinMax ccc 0.0 256.0
_______________________


In [6]:
# galaxies in the cluster 29176 at nout=187
gal_this_cluster = [gal.data[0] for gal in prg_final if gal.cluster == 29176]
gal_this_cluster = np.hstack(gal_this_cluster)

In [ ]:
clu = hcat.data[np.argmax(hcat.data["np"])]
rscale = 3.0

cx,cy,cz = clu["x"], clu["y"], clu["z"]
x,y,z = hcat.data["x"], hcat.data["y"], hcat.data["z"]
dd = np.sqrt(np.square(cx - x) + np.square(cy - y) + np.square(cz - z))

i_cluster = np.where((dd < (clu["rvir"]*rscale)) * (hcat.data["mvir"] > 1e10))[0]

gcat = hmo.Halo(base=wdir_clu, nout=187, is_gal=True)

ax = draw.pp.pp_halo(hcat, 400, ind=i_cluster, color_field="level", color_log=False)
#i_good_gal = np.where(gcat.data["mvir"] > 5e9)[0]
i_good_gal = mtc.match_list_ind(gcat.data["id"], gal_this_cluster["id"]) #Only the galaxies in the final mpgs list.
ax = draw.pp.pp_halo(gcat, 400, ind=i_good_gal, edgecolor='green')#color_field="level", color_log=False)
plt.show()

Match galaxies and level 1 halos? 
-> 


Use tree, but also use halo catalog for halo level information.

In [8]:
import tree.treemodule as tmo
gt = tmo.load_tree(wdir_clu, is_gal=True)
ht = tmo.load_tree(wdir_clu, is_gal=False)

Loaded an extended tree
Loaded an extended tree


Every galaxy has a tree. But 20 galaxies have failed to find match. 

In [10]:
i_good_gt = mtc.match_list_ind(gt.data["id"], gal_this_cluster["id"])


dts = [3,5]

info = load.info.Info(187, base=wdir_clu)
good_hals, good_gals = gal_hal_pair(cluster, gt, ht, info,
                                     ids = gal_this_cluster["id"],
                                     dominant = 0.4,
                                     abs_tol_pos = 1e-2,
                                     abs_tol_vel = 100,
                                     nout_fi = 187,
                                     dts = dts)

Out of 150 galaxies, matched 130 galaxies.


In [42]:
missing = np.setdiff1d(gal_this_cluster["id"], good_gals["Orig_halo_id"])
print("Galaxies faield to match are:", missing)

Galaxies faield to match are: [  4 116 133 137 199 213 221 227 267 274 314 321 341 348 397 685 702 706
 708 753]


In [11]:
ax = draw.pp.pp_halo(hcat, 400, ind=i_cluster, edgecolor="blue")#color_field="level", color_log=False)
i_good_gal = mtc.match_list_ind(gcat.data["id"], gal_this_cluster["id"]) #Only the galaxies in the final mpgs list.
ax = draw.pp.pp_halo(gcat, 400, ind=i_good_gal, edgecolor='green')#color_field="level", color_log=False)
i_good_gal = mtc.match_list_ind(gcat.data["id"], good_gals["Orig_halo_id"]) #Only the galaxies in the final mpgs list.
draw.pp.pp_halo(gcat, 400, ind=i_good_gal, edgecolor='red', ax=ax)
plt.show()

In [37]:
fig, ax = plt.subplots()


draw.pp.pp_halo(good_hals, 400, edgecolor='blue', rscale=1e-3, ax=ax)
#ax.pp_hal_meta.region["zr"] = (0, 1e9)
#ax.pp_hal_meta.region["ranges"][2] = (0, 1e9)
print("_______________________")
draw.pp.pp_halo(good_gals, 400, edgecolor='green', rscale=1e-3, ax=ax)
ax.set_aspect('equal', 'datalim')
plt.show()

[Halo.load_info] loading info
[Halo.load_info] nout = 187, base =/home/hoseung/Work/data/29176/
MinMax ccc 0.0 256.0


ValueError: min() arg is an empty sequence

In [ ]:
from analysis.misc import load_cat
cat_final = load_cat("/home/hoseung/Work/data/29176/easy_final/catalog187.pickle")
# some galaxies have erroneous values.
cat_final["idx"][cat_final["idx"] < 0] = -1

In [ ]:
def get_comp_dist(hal_now, gal, nreturn=5, mcut=True):
    """Measure 6D distance and return Nreturn closest entries"""
    if mcut:
        hal_now = hal_now[hal_now["m"] > gal["mstar"]]
    dd = dist(hal_now, gal)
    vv = distv(hal_now, gal)
    dd_q1 = np.percentile(dd, 10)
    vv_q1 = np.percentile(vv, 10)
    comp_dist = np.sqrt(np.square(dd/dd_q1) + np.square(vv/vv_q1))
    ind_sort = np.argsort(comp_dist)
    return comp_dist[ind_sort[:nreturn]], hal_now[ind_sort[:nreturn]]


hal_ids = list(hcat.data["id"])
hal_gal_pair=np.zeros(len(cat_final), dtype=([("gid", "<i8"), ("hid", "<i8")]))
for i, gal in enumerate(cat_final[np.argsort(cat_final["mstar"])[::-1]]):
    # print(np.log10(gal["mstar"]))
    comp_dist_this, good_hals_this = get_comp_dist(hcat.data, gal, nreturn=10, mcut = True)
                # halo must be more massive than the galaxy.
    # print(good_hals_this["m"])
    # Halo mass is exclusive, while the BCG stellar mass may include much more.
    good_hals_this = good_hals_this[good_hals_this["m"] > gal["mstar"]]
    
    matched_hal = good_hals_this[0]["id"]
    # If everything is good. 
    hal_gal_pair["gid"] = gal["id"]
    hal_gal_pair["hid"] = hal_ids.pop(hal_ids.index(matched_hal))
    print(matched_hal)

In [ ]:
fig, axs = plt.subplots(1,2)
gid_in_clu = np.array([gal.data['id'][0] for gal in prg_final if gal.cluster == cluster])

ghal = hmo.Halo(base = wdir_clu, nout=187, is_gal=True)

i_final_sample = mtc.match_list_ind(ghal.data["id"], gid_in_clu)
axs[1] = draw.pp.pp_halo(ghal.data, 400, color_log=False, radius="rvir")
#draw.pp.pp_halo(hcat.data, 400, color_log=False, radius="rvir", ax=ax, edgecolor='red')
draw.pp.pp_halo(ghal.data[i_final_sample], 400, ax=axs[1], radius="rvir", edgecolor="blue")
print(axs[1].pp_hal_meta.region)
draw.pp.pp_halo(ghal.data, 400, ax=axs[0], region=axs[1].pp_hal_meta.region, radius="rvir")
#axs[0].set_xlim(axs[1].get_xlim())
#axs[0].set_ylim(axs[1].get_ylim())
plt.show()

In [ ]:
max(ghal.data[i_final_sample]["x"])

In [ ]:
axs[0].pp_hal_meta.region